### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [3]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000165685FACB0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000165685F99F0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [6]:
result

AIMessage(content='Here are a few ways to translate "Hello, how are you?" to French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Bonjour, comment vas-tu ?** (Informal, used with someone you know well)\n* **Salut, ça va ?** (Very informal, used with friends) \n\n\nThe best option depends on the context and your relationship with the person you are speaking to. \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 21, 'total_tokens': 114, 'completion_time': 0.169090909, 'prompt_time': 0.000156469, 'queue_time': 0.013892930000000001, 'total_time': 0.169247378}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-073e8cab-9676-43ac-81c5-90967f62f5af-0', usage_metadata={'input_tokens': 21, 'output_tokens': 93, 'total_tokens': 114})

In [7]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Here are a few ways to translate "Hello, how are you?" to French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Bonjour, comment vas-tu ?** (Informal, used with someone you know well)\n* **Salut, ça va ?** (Very informal, used with friends) \n\n\nThe best option depends on the context and your relationship with the person you are speaking to. \n'

In [8]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'Here are a few ways to translate "Hello, how are you?" to French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most formal option, used with strangers or people you respect.)\n\n**Informal:**\n\n* **Salut, comment vas-tu ?** (This is used with friends and family.)\n* **Coucou, ça va ?**  (This is very informal and friendly.)\n\nLet me know if you\'d like other variations!\n'

In [9]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [10]:
result=prompt.invoke({"language":"French","text":"Hello"})

In [11]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [12]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'